In [1]:
include("NTKernel.jl")

kernel (generic function with 1 method)

In [1]:
# PSUDEO data
g = x -> cos(2x)+sin(x/2)  # λ function

X = range(1, stop=12, length=100)
X = Matrix(reshape(X, 1, :))

Y = map(g,X)

1×100 Matrix{Float64}:
 0.0632787  -0.0789055  -0.192898  …  -0.390548  -0.118139  0.144764

In [30]:
# nodes on hidden layer
Nh_list = [300] 
models = []
activation = sigmoid

InputDim = 1

for Nh in 1:length(Nh_list)
    push!(models,Chain(Dense(InputDim=>Nh_list[Nh],activation),Dense(Nh_list[Nh]=>InputDim))|>f64)
end

Kernels = []
for model in models
    push!(Kernels, kernel(model, X))
end

Kernels_Norms = []

loss(a, b) = Flux.Losses.mse(models[1](a), b)
epoch = 1

for i = 1:epoch
    Flux.train!(loss, Flux.params(models[1]), [(X,Y)], Descent(0.01))
end

for (i, K) in enumerate(Kernels)
    Nh = Nh_list[i] # Get the corresponding Nh for this kernel
    normalized_K = K / sqrt(Nh) # Normalize the kernel matrix
    push!(Kernels_Norms, normalized_K) # Store the normalized kernel
end

λ_values = []
for K in Kernels_Norms
    push!(λ_values, eigen(K).values) # Compute eigenvalues for each kernel and store
end


λ_values

1-element Vector{Any}:
 [-7.175648685628797e-14, -2.5757174171303632e-14, -2.353672812205332e-14, -1.5987211554602254e-14, -1.4210854715202004e-14, -7.993605777301127e-15, -3.946591350488445e-15, -3.898185820173079e-15, -3.552713678800501e-15, -3.552713678800501e-15  …  1.3322676295501878e-14, 4.3520742565306136e-14, 5.293081658439716e-14, 3.2179953460635924e-12, 2.5678080395837354e-9, 4.3056373272110614e-7, 3.341073164880052e-5, 0.036786492125568164, 9.952570791136427, 478.5798694070651]

In [2]:
using Plots

Ŷ = []
for i in 1:length(X)
    push!(Ŷ, models[1](X[:, i])[1])  # Assuming models[1] outputs a 1-element array
end

scatter(X[:], Y[:], label="Real Data")
scatter!(X[:], Ŷ[:], label="Predicted Data")


UndefVarError: UndefVarError: `models` not defined